In [1]:
from dotenv import load_dotenv
# OPEN AI API 키 설정
load_dotenv()

True

In [2]:
import os

def project_setup(name):
    os.environ['LANGSMITH_PROJECT'] = name
    os.environ['LANGSMITH_TRACING'] = 'true'
    print('프로젝트: {} 추적 시작'.format(name))

def stop_tracing():
    os.environ['LANGSMITH_TRACING'] = 'false'
    print('LangSmith 추적 중지')

project_setup('LangChain Test')

프로젝트: LangChain Test 추적 시작


In [ ]:
from langchain_openai import ChatOpenAI

model_name = "gpt-4o"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048, # 최대 토큰수
    model_name=model_name,  # 모델명
) #.bind(logprobs=True)

In [4]:
# https://spri.kr/posts/view/23669

In [5]:
from langchain_core.documents import Document

document = Document("안녕하세요? 이건 랭체인의 도큐먼드 입니다")

In [6]:
# 도큐먼트의 속성 확인
document.__dict__

{'page_content': '안녕하세요? 이건 랭체인의 도큐먼드 입니다', 'metadata': {}, 'type': 'Document'}

In [7]:
# 메타데이터 추가
document.metadata["source"] = "TeddyNote"
document.metadata["page"] = 1
document.metadata["author"] = "Teddy"

#### PyPDFLoader

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
# 예제 파일 경로
FILE_PATH = "./data/SPRI_AI_Brief_2023년12월호_F.pdf"

In [9]:
# %pip install -qU pypdf

In [10]:
from langchain_community.document_loaders import PyPDFLoader

# 로더 설정
loader = PyPDFLoader(FILE_PATH)

In [ ]:
# PDF 로더
docs = loader.load()

# 로드된 문서의 수 확인
len(docs)

In [ ]:
# 첫번째 문서 확인
docs[0]

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 문열 분할기 설정
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
# 문서 분할
docs = loader.load_and_split(text_splitter=text_splitter)

# 로드된 문서의 수 확인
len(docs)
# 첫번째 문서 확인
docs[0]

In [ ]:
# generator 방식으로 문서 로드
for doc in loader.lazy_load():
    print(doc.metadata)

#### JSONLoader

In [9]:
%pip install -qU jq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# https://gist.githubusercontent.com/raine/da15845f332a2fb8937b344504abfbe0/raw/37cb62958ebf41ce931c7236b5453838a1f17d9c/people.json

In [ ]:
from pprint import pprint
from langchain_community.document_loaders import JSONLoader

# JSONLoader 생성
loader = JSONLoader(
    file_path=r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\data\vectorized_data.json",
    jq_schema=".[].name",
    text_content=False,
)

# 문서 로드
docs = loader.load()

# 결과 출력
pprint(docs)

In [ ]:
%pip install faiss-cpu
%pip install faiss-gpu

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader(r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\data\supplement.json")
# loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
# split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1)

42787

In [8]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

# 임베딩
embeddings = OpenAIEmbeddings()
batch_size = 100

# 1. texts 추출
texts = [doc.page_content for doc in split_doc1]
metadatas = [doc.metadata for doc in split_doc1]

# 2. 벡터를 나눠서 처리
vectors = []
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    vectors.extend(embeddings.embed_documents(batch))

KeyboardInterrupt: 

In [ ]:
dimension_size = 1536

# FAISS 벡터 저장소 생성
db = FAISS(
    embedding_function=OpenAIEmbeddings(),
    index=faiss.IndexFlatL2(dimension_size),
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [ ]:
# DB 생성
db = FAISS.from_documents(documents=split_doc1, embedding=OpenAIEmbeddings())

In [13]:
import json
import faiss
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain_community.docstore.in_memory import InMemoryDocstore

# ----- 설정 -----
JSON_PATH = r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\data\vectorized_data.json"         # 원본 벡터 JSON
SAVE_DIR = "faiss_index"                   # 저장 폴더
DIM = 1536                                 # 벡터 차원 (gpt-3.5-turbo 기준)

# ----- 1. JSON 로드 -----
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

texts = [item["text"] for item in raw_data]
metadatas = [item["metadata"] for item in raw_data]
vectors = [item["embedding"] for item in raw_data]

# ----- 2. Document 객체 생성 -----
documents = [
    Document(page_content=texts[i], metadata=metadatas[i])
    for i in range(len(texts))
]

# ----- 3. FAISS 인덱스 생성 -----
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings

dim = len(vectors[0])
index = faiss.IndexFlatL2(dim)
vectorstore = FAISS(
    embedding_function=OpenAIEmbeddings(),  # 이건 이후 검색용
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# ----- 4. 저장 -----
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
vectorstore.save_local(SAVE_DIR)
print(f"✅ 저장 완료: {SAVE_DIR}/index.faiss, index.pkl")


✅ 저장 완료: faiss_index/index.faiss, index.pkl


In [27]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
db = FAISS.load_local(r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\faiss_index", embeddings, allow_dangerous_deserialization=True)


In [28]:
# 문서 저장소 ID 확인
db.index_to_docstore_id

{0: '02d61f88-65f3-4a6e-bd05-6ee0a0ab9735',
 1: '3de8977d-6474-443f-8594-f38fd0aad665',
 2: 'b43d004c-0025-440e-bdc3-b05bf1150647',
 3: '5c29f76e-0ff3-4ddd-83a9-dbfcdce3c9de',
 4: '58cd9347-3a45-4fcb-884d-90a846c973a2',
 5: 'b10f1044-f6be-4494-ac30-913cd199ef12',
 6: '6c3caf98-cbe6-4779-acd4-4a7155dfd4f2',
 7: 'fa6206e5-b83a-477a-8a07-8a36170e1be8',
 8: 'a0adc18e-5317-4e6f-91aa-913f3bddd752',
 9: '39e476a3-9d66-4d52-adaf-c41193dec5ac',
 10: '53ed452e-979f-4665-a099-61dc09da59c0',
 11: '5dce34b7-b65c-4507-963d-8f92f041b108',
 12: 'fdb6edf2-5f3e-4453-812e-f843fc898d66',
 13: 'e7d7f9cd-5c21-4a00-82d6-cb6ecc202be0',
 14: '4ac812f6-3366-43c8-94f3-1620be12f975',
 15: 'bccf818c-df49-47ae-8c23-c8ce5e778728',
 16: '3ea7437e-585d-4860-be64-3c038176b77c',
 17: 'c6e6f562-41e3-4fd0-9a9e-a3820cb53ea6',
 18: '94242ad3-b917-47ec-9bdf-455c6638a0aa',
 19: 'aa615a04-7bcc-4247-b1ad-d70fa87232bd',
 20: 'f0ac621e-686b-413d-ab64-367024fdd018',
 21: 'a3b73683-2123-4dc0-9611-6c68a62e6903',
 22: '6c0d183e-c929-

In [29]:
# 저장된 문서의 ID: Document 확인
db.docstore._dict

{'02d61f88-65f3-4a6e-bd05-6ee0a0ab9735': Document(page_content='[비타민E]①유해산소로부터 세포를 보호하는데 필요\n알레르기 체질이신 경우 성분을 확인하신 후 섭취하여 주시기 바랍니다.\n주요 성분: 비타민E, d-알파-토코페롤, 대두유', metadata={'PRDLST_REPORT_NO': 2004002000197, 'BSSH_NM': '풀무원건강생활(주)', 'PRDLST_NM': '토코페롤비타민E', 'DISPOS': '이미 이취가 없고 고유의 향미가 있는 미황색 유상 내용물을 함유하고 있는 투명한 타원형 연질캡슐', 'CSTDY_MTHD': '직사광선을 피하고 서늘한 곳에 보관', 'POG_DAYCNT': '제조일로부터 24개월', 'NTK_MTHD': '1일 1회, 1회 1캅셀을 물과 함께 섭취', 'IFTKN_ATNT_MATR_CN': '알레르기 체질이신 경우 성분을 확인하신 후 섭취하여 주시기 바랍니다.', 'RAWMTRL_NM': '비타민E', 'INDIV_RAWMTRL_NM': 'd-알파-토코페롤', 'ETC_RAWMTRL_NM': '대두유', 'CAP_RAWMTRL_NM': '젤라틴, 글리세린', 'PRIMARY_FNCLTY': '[비타민E]①유해산소로부터 세포를 보호하는데 필요'}),
 '3de8977d-6474-443f-8594-f38fd0aad665': Document(page_content='유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음\r\n유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음\n고온다습한 환경에서의 사용을 가급적 삼간다.\n주요 성분: 프로바이오틱스(원료성), Lactobacillus acidophilus분말(가루, 과립)(생균으로 100,000,000,000개/g 이상, 보호제 및 희석제 적량), Lactobacillus rhamnosus분말(가루, 과립)(생균으로 100,000,000,000개/g 이상, 보호제 및 희석제 적량), B

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# TextLoader를 사용하여 파일을 로드합니다.
loader = TextLoader(r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\data\supplement.json")

# 문서를 로드합니다.
documents = loader.load()

# 문자 기반으로 텍스트를 분할하는 CharacterTextSplitter를 생성합니다. 청크 크기는 300이고 청크 간 중복은 없습니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 로드된 문서를 분할합니다.
split_docs = text_splitter.split_documents(documents)

# OpenAI 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# 분할된 텍스트와 임베딩을 사용하여 FAISS 벡터 데이터베이스를 생성합니다.
db = FAISS.from_documents(split_docs, embeddings)

KeyboardInterrupt: 

In [59]:
# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당
retriever = db.as_retriever()

In [ ]:
# 관련 문서를 검색
docs = retriever.invoke("임베딩(Embedding)은 무엇인가요?")

for doc in docs:
    print(doc.page_content)
    print("=========================================================")

In [26]:
import json
import faiss
import uuid
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
import numpy as np

# ----- 설정 -----
JSON_PATH = r"C:\Users\bona_\Desktop\git_pill_addict\Pill-addict\kahee\app\data\vectorized_data.json"   # 임베딩된 벡터 + 문서 파일
SAVE_DIR = "faiss_index"             # 저장할 디렉토리 이름
DIM = 1536                           # 벡터 차원수 (gpt-3.5-turbo 기준)

# ----- 1. JSON 파일 읽기 -----
with open(JSON_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

texts = [item["text"] for item in raw_data]
metadatas = [item["metadata"] for item in raw_data]
vectors = [item["embedding"] for item in raw_data]
ids = [str(uuid.uuid4()) for _ in raw_data]  # 문서 ID

# ----- 2. Document 객체 생성 -----
documents = [
    Document(page_content=texts[i], metadata=metadatas[i])
    for i in range(len(texts))
]

# ----- 3. FAISS 인덱스 초기화 -----
index = faiss.IndexFlatL2(DIM)
index.add(np.array(vectors).astype("float32"))

docstore = InMemoryDocstore(dict(zip(ids, documents)))
index_to_docstore_id = {i: ids[i] for i in range(len(ids))}

vectorstore = FAISS(
    embedding_function=OpenAIEmbeddings(),  # 검색 시 사용
    index=index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)


# ----- 5. 저장 -----
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
vectorstore.save_local(SAVE_DIR)
print(f"✅ 저장 완료: {SAVE_DIR}/index.faiss & index.pkl")


✅ 저장 완료: faiss_index/index.faiss & index.pkl


In [21]:
pip install -U langchain langchain-community


  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 12.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 24.1 MB/s eta 0:00:00
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.53
    Uninstalling langchain-core-0.1.53:
      Successfully uninstalled langchain-core-0.1.53
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.4
    Uninstalling langchain-text-splitters-0.2.4:
      Successfully uninstalled langchain-text-splitters-0.2.4
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Successfully uninstalled langchain-0.2.1
  Attempting

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.59 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import time
import os
import psutil
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 경로 설정
INDEX_DIR = "faiss_index"  # 이미 만들어진 index.faiss + index.pkl 폴더
QUERY = "임산부가 먹으면 안 되는 영양제는?"

# 메모리 측정 함수
def get_memory_usage_mb():
    return psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024

# 초기 상태
print("▶ 초기 메모리 상태")
print(f"  메모리 사용량: {get_memory_usage_mb():.2f} MB\n")

# 로딩 테스트
print("▶ FAISS 인덱스 로딩 테스트")
start_time = time.time()
db = FAISS.load_local(INDEX_DIR, OpenAIEmbeddings(), allow_dangerous_deserialization=True)
load_time = time.time() - start_time
print(f"  로딩 시간: {load_time:.2f}초")
print(f"  메모리 사용량: {get_memory_usage_mb():.2f} MB\n")

# 검색 테스트
print("▶ 검색 테스트")
start_time = time.time()
results = db.similarity_search(QUERY, k=3)
search_time = time.time() - start_time
print(f"  검색 시간: {search_time:.2f}초")
print(f"  메모리 사용량: {get_memory_usage_mb():.2f} MB\n")

# 검색 결과 출력
for i, doc in enumerate(results, 1):
    print(f"[{i}] {doc.page_content[:100]}...")


▶ 초기 메모리 상태
  메모리 사용량: 2891.97 MB

▶ FAISS 인덱스 로딩 테스트
  로딩 시간: 1.11초
  메모리 사용량: 3102.54 MB

▶ 검색 테스트
  검색 시간: 0.77초
  메모리 사용량: 3104.67 MB

[1] 과체중인 성인의 체지방 감소에 도움을 줄 수 있습니다
① 위장장애가 발생할 수 있습니다.
② 영유아, 임산부는 섭취를 삼가야 합니다.
③ 식사조절, 운동을 같이 하는 것이 바람직합...
[2] 수면에 도움을 줄 수 있음
영·유아, 어린이, 임산부 및 수유부는 섭취에 주의 
의약품(항히스타민제 또는 중추신경계억제제)과 병용섭취 시 주의 
사용하다 남은 제품은 미생물 오염이...
[3] ①과체중인 성인의 체지방 감소에 도움을 줄 수 있음
①항산화 작용을 하여 유해산소로부터 세포를 보호하는데 필요
특정질환, 특이체질, 알레르기체질, 임산부의 경우에는 간혹 개인에 따...
